**Dask** is the best and fastest one for reading a 5GB file.</br>
**Ray** is much slower and inefficient that dask.</br>
**Modin** is just lib which its core is Ray or Dask. You can specify manually. </br>
Working with pandas to read the file, need a little tweaks to do. But its not a good way to load the file especially when  your workflow require slicing, manipulating, exporting</br>


In [1]:
! pip install -q kaggle
!python -m pip install "dask[complete]"
!pip install ray
!pip install modin
!pip install pyaml
!mkdir ~/.kaggle
!ls 
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

! kaggle competitions download -c 'new-york-city-taxi-fare-prediction'
!unzip /content/new-york-city-taxi-fare-prediction.zip
! du /content/train.csv


     |████████████████████████████████| 837 kB 5.3 MB/s 
     |████████████████████████████████| 136 kB 18.9 MB/s 
     |████████████████████████████████| 830 kB 41.2 MB/s 
     |████████████████████████████████| 822 kB 17.9 MB/s 
     |████████████████████████████████| 802 kB 48.4 MB/s 
     |████████████████████████████████| 802 kB 42.8 MB/s 
     |████████████████████████████████| 793 kB 47.7 MB/s 
     |████████████████████████████████| 793 kB 44.0 MB/s 
     |████████████████████████████████| 791 kB 43.8 MB/s 
     |████████████████████████████████| 786 kB 37.0 MB/s 
     |████████████████████████████████| 779 kB 54.9 MB/s 
     |████████████████████████████████| 778 kB 42.9 MB/s 
     |████████████████████████████████| 776 kB 37.5 MB/s 
     |████████████████████████████████| 769 kB 37.3 MB/s 
     |████████████████████████████████| 766 kB 45.0 MB/s 
     |████████████████████████████████| 1.0 MB 42.7 MB/s 
     |████████████████████████████████| 722 kB 44.8 MB/s 
     |█████████

In [2]:

import logging
import os
import subprocess
import yaml
import pandas as pd
import datetime 
import gc
import re
import numpy as np
# import ray
# ray.init(ignore_reinit_error=True)
# import modin.pandas as pd
import dask.dataframe as dd

In [ ]:

# df = pd.read_csv('/content/train.csv')
# df.head()

In [3]:

df = dd.read_csv('/content/train.csv')
df.head()

,key,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
0,2009-06-15 17:26:21.0000001,4.5,2009-06-15 17:26:21 UTC,-73.844311,40.721319,-73.841610,40.712278,1
1,2010-01-05 16:52:16.0000002,16.9,2010-01-05 16:52:16 UTC,-74.016048,40.711303,-73.979268,40.782004,1
2,2011-08-18 00:35:00.00000049,5.7,2011-08-18 00:35:00 UTC,-73.982738,40.761270,-73.991242,40.750562,2
3,2012-04-21 04:30:42.0000001,7.7,2012-04-21 04:30:42 UTC,-73.987130,40.733143,-73.991567,40.758092,1
4,2010-03-09 07:51:00.000000135,5.3,2010-03-09 07:51:00 UTC,-73.968095,40.768008,-73.956655,40.783762,1


# Create YAML File

In [4]:
toYaml = {
    "path": "/content/train.csv",
    "columns": list(df.columns),
    "#rows" : df.shape[0].compute(),
    "#cols" : df.shape[1],
    "inputSep" : "," ,
    "outputSep" : "|", 
}


with open('correct.yml', 'w+') as f:
  yaml.dump(toYaml, f, allow_unicode=True)



toYaml = {
    "path": "/content/train.csv",
    "columns": list(df.columns[:-1]),
    # "#rows" : df.shape[0].compute(),
    "#cols" : df.shape[1],
    "inputSep" : "," ,
    "outputSep" : "|", 
}


with open('wrong.yml', 'w+') as f:
  yaml.dump(toYaml, f, allow_unicode=True)


# Utility Functions

In [7]:
def load_yaml(path='/content/correct.yml'):
  try:
    with open(path, 'r+') as f:
      fromYaml = yaml.safe_load(f)
    return fromYaml
  except OSError as e:
    logging.error(e)
  except yaml.YAMLError as exc:
    logging.error(exc)
  

def validator(config, df, check_rows=False):
  df.columns = df.columns.str.lower()
  expected_cols = np.array(list(map(lambda x: x.lower(),  config['columns'])))
  df_cols = np.array(list(df.columns))
  if len(df_cols) == config['#cols'] and (expected_cols  == df.columns).all():
    print(expected_cols)
    print(df_cols)
    print("column name and column length validation passed")
    if check_rows:
      if df.shape[0].compute() == config['#rows']:
        print("#Rows validation passed")
        return 1
      else:
        return 0
  else:
    print("column name and column length validation failed")
    mismatched_columns_file = list(set(df.columns).difference(expected_cols))
    print("Following File columns are not in the YAML file",mismatched_columns_file)
    missing_YAML_file = list(set(expected_cols).difference(df.columns))
    print("Following YAML columns are not in the file uploaded",missing_YAML_file)
    logging.info(f'df columns: {df.columns}')
    logging.info(f'expected columns: {expected_cols}')
    return 0

def load_df(config):
  df = dd.read_csv(config['path'], sep=config['inputSep'])
  return df



# Run

In [8]:
config = load_yaml()
df = load_df(config)
df.head()
validator(config, df, check_rows=True)

['key' 'fare_amount' 'pickup_datetime' 'pickup_longitude'
 'pickup_latitude' 'dropoff_longitude' 'dropoff_latitude'
 'passenger_count']
['key' 'fare_amount' 'pickup_datetime' 'pickup_longitude'
 'pickup_latitude' 'dropoff_longitude' 'dropoff_latitude'
 'passenger_count']
column name and column length validation passed


1

# Write file to gz format

In [ ]:
df.to_csv('./output.csv.gz', header=True, index=False, sep=config['outputSep'], compression='gzip')

      